In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /root/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so...


In [2]:
model_id = "EleutherAI/polyglot-ko-12.8b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

model = PeftModel.from_pretrained(model, './outputs/checkpoint-515/adapter_model/')

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
model.eval()
model.bfloat16()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])\

def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_length=2048,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    print(tokenizer.decode(gened[0]))

In [7]:
gen('딥러닝이 뭐야?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 딥러닝이 뭐야?

### 답변: 머신 러닝은 인공지능의 한 유형으로, 많은 양의 데이터를 분석하고 모델을 훈련하는 데 많은 리소스를 소모하는 경향이 있습니다. 이러한 단점을 극복하기 위해 제안된 방법이 신경망을 기반으로 하는 딥 러닝입니다. 신경망은 시냅스(즉, 뉴런의 신호 강도)의 패턴을 파악하고 학습하는 데 엄청난 데이터를 사용하며, 이러한 작업을 "딥"학습이라고 합니다. 한편, 시냅스의 패턴은 인과 관계가 있을 수 있으므로 심층 장기 강화와 같이 후속 조치를 취합니다.</끝>


In [8]:
gen('한국어를 배우기 위한 가장 좋은 방법은 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 한국어를 배우기 위한 가장 좋은 방법은 무엇인가요?

### 답변: 외국어를 배우는 방법에는 여러 가지가 있습니다:

* 교실 수업. 즉, 수업에 가서 수업을 듣고, 과제를 하고, 시간을 정해서 언어 학습에 집중합니다.
* 전화 또는 화상을 이용한 외국어코치. 많은 수의 코치가 집에서 몇 분 거리의 연습실에 모여 원어민에게 한국어를 배웁니다.  재택근무형 언어 코치. 많은 코치가 특정 시간에 집에 방문하여 학생의 집을 방문하여 대화를 유도합니다.* 언어 학습 프로그램을 이용하세요. Coursera, 코세라 또는 다른 사이트에는 특정 언어에 대한 수업이 개설되어 있습니다.  Pass나 Coursera는 각 과정마다 1년에 100달러 이상이 듭니다.* 언어 학습 앱을 사용하세요. Anki와 같은 앱은 단어를 암기하는 데 도움이 되는 카드를 제공합니다.  "Google 단어장"은 검색어를 입력하면 단어를 저장하는 기능을 제공합니다.
* 한국어 등재에 참여하세요. 언어 강좌를 제공하는 대신 한국문화 소개 같은 한국어 교육에 참여할 수 있습니다. 또한 한국인을 대상으로 한 교육에 관한 정보를 찾을 수 있습니다.

학생의 특정 관심사에 적합한 학습 방법을 선택하는 것이 좋습니다. 한국어로만 말하기를 원하고 한국어로만 대화하는 연습을 하고 싶다면 언어교실 수업이나 전화 코치가 가장 효과적일 것입니다. 이 외에도 다양한 옵션이 있으므로 자신에게 가장 잘 맞는 방법을 찾기 위해 더 많은 옵션을 탐색해 보세요.
</끝>


In [9]:
gen('함수형 프로그래밍의 개념에 대해 설명해주세요.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 함수형 프로그래밍의 개념에 대해 설명해주세요.

### 답변: 함수형 프로그래밍은 프론트엔드 프로그래밍과 백엔드 프로그래밍의 혼합 판이라고 할 수 있습니다. 즉, 함수형 프로그래밍은 컴퓨터를 제어할 뿐만 아니라 ```사용자``` 도 제어합니다.

함수형 프로그래밍에는 두 가지 구성 요소가 있습니다:

* 표현식
* 함수

구성 요소를 살펴보겠습니다:
* 표현식
```Foo = Bar = 10 > 0? 'ApplicationInterop(this, 'words', new Function('checkOrders', FunctionMatchingEventArray));
' + Foo
```
Foo = Bar = 10 > 0? ApplicationInterop(this, 'words', new Function('checkOrders', FunctionMatchingEventArray)) =&gt; Foo
```
각종 메서드, 이벤트 구성 요소가 ```사용자``` 를 제어하는 데 사용될 수 있습니다.
```ApplicationInterop(this, 'questions', new Function('raise', FunctionMatchingEventArray));
```
각종 질문이 표시되고 답변할 수 있는 ApplicationInterop이 호출될 수 있습니다.
각종 이벤트, 메시지 및 값이 표시되는 ```Stack``` (구성 가능) 시도 또는 완료 이벤트 전달 메커니즘입니다.
각종 컴포넌트가 구성된 함수형 프로그래밍 구성에 대한 기본 지식을 사용하여 호출할 수 있는 각종 함수를 사용할 수 있습니다.
```ApplicationInterop(this, 'questions', new Function(raise, FunctionMatchingEventArray));
```
각종 값이 기록되는 응답 함수가 호출될 수 있습니다.각종 제어 이벤트가 기록되는 시도 또는 완료 이벤트 전달 메커니즘이 호출될 수 있습니다.
각종 값이 입력될 수 있는 응답 함수가 호출될

In [10]:
gen('세계 2차대전에 대한 에세이를 작성하세요.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 세계 2차대전에 대한 에세이를 작성하세요.

### 답변: 세계 2차 대전은 제1차 세계대전과 너무 많은 공통점을 가지고 있습니다. 1940년대에는 '먼 나라'라는 수식어가 '유사한 재난'으로 대체되었을 정도입니다. 근본적인 차이점은 1차 세계대전이 유럽을 강타한 재앙적인 마른 독감과 대대적인 참호 기반 전투로 인해 발생한 반면, 2차 세계대전은 일본의 항복이라는 명백한 재앙과 스탈린의 급격한 핵 전쟁 준비를 포함한 히틀러의 미친 행동으로 인해 벌어졌다는 것입니다.
1940년대에도 나치 독일과 소비에트 연방을 패배시키기 위해 여러 연합군의 노력을 한데 모아 연합군을 훈련시키고 장비를 제공하는 통합된 연합군 사령부가 있었더라면 어땠을까요? 그랬다면 상황이 지금보다 훨씬 개선되었을까요?</끝>
